In [2]:
import pandas as pd
import numpy as np
import re
import tqdm
from bs4 import BeautifulSoup
import time
import matplotlib.pyplot as plt

In [22]:
# この文字列ありますか？関数
def search_object(search,object):
    return bool(re.search(search,object))

In [613]:
# csvファイルごとに抜き出してあとでくっつける
num_files = 1237
finish_file = 1000
suumo_details = pd.DataFrame()
for num in tqdm.tqdm(range(finish_file+1,num_files+finish_file+1)):
    file_path = f'./soup_tank/soup_tank_{num * 100}.csv'
    suumo_hundred = pd.read_csv(file_path)

    # soupから情報を抜き出す
    data_rooms_all = []
    for url,html in zip(suumo_hundred['url'],suumo_hundred['soup']):
        search = '(<h1 class="section_h1-header-title">)(.*)(</h1>)'
        is_exist = search_object(search,html)
        if is_exist :

            soup_room = BeautifulSoup(html, 'html.parser')

            # 部屋情報の器
            data_room = []

            # 物件名
            name = soup_room.find(class_='section_h1-header-title').text
            data_room.append(name)
            # 費用に関する情報
            colname = ['家賃','管理費・共益費','敷金','礼金','保証金','敷引・償却']
            cost_info = soup_room.find_all(class_='property_view_note-list')
            for mother in cost_info:
                for child in mother.find_all('span'):
                    data_room.append(child.text)
            # 基本情報
            colname = ['住所','駅徒歩','間取り','面積','築年数','階','向き','建物種別']
            basic_info = soup_room.find_all(class_='property_view_table-body')
            for k, (name, info ) in enumerate(zip(colname,basic_info)):
                if k == 1:
                    data_room.append(info.find('div').text)
                else:
                    data_room.append(info.text)
            # オプション情報
            option = soup_room.find_all(class_='inline_list')
            data_room.append(option[1].text)
            # 概要情報
            colname = ['間取り詳細','構造','階建','築年月','損保','駐車場','入居','取引態様',
                       '条件','取り扱い店舗物件コード','SUUMO物件コード','総戸数',
 #                      '情報更新日','次回更新日','仲介手数料','保証会社','ほか初期費用'
                      ]
            overview = soup_room.find(class_='data_table table_gaiyou')
            children = overview.find_all('td')
            for name, info in zip(colname,children[:12]):
                data_room.append(info.text.strip())
            # url
            data_room.append(url)
            # 部屋情報を器に入れる
            data_rooms_all.append(data_room)
    # 100ごとに部屋情報を器に入れる
    colnames = ['物件名','家賃','管理費・共益費','敷金','礼金','保証金','敷引・償却',
                '住所','駅徒歩','間取り','面積','築年数','階','向き','建物種別',
                'オプション',
                '間取り詳細','構造','階建','築年月','損保','駐車場','入居','取引態様',
                '条件','取り扱い店舗物件コード','SUUMO物件コード','総戸数',
   #             '情報更新日','次回更新日','仲介手数料','保証会社','ほか初期費用',
                'url']
    suumo_details_tmp = pd.DataFrame(data_rooms_all,columns=colnames)
    suumo_details = pd.concat([suumo_details, suumo_details_tmp], axis=0)
    
suumo_details = suumo_details.reset_index(drop=True)


100%|██████████| 1237/1237 [7:33:57<00:00, 22.02s/it]  


In [614]:
suumo_details

,物件名,家賃,管理費・共益費,敷金,礼金,保証金,敷引・償却,住所,駅徒歩,間取り,...,築年月,損保,駐車場,入居,取引態様,条件,取り扱い店舗物件コード,SUUMO物件コード,総戸数,url
0,コンフォルト若宮,10.5万円,管理費・共益費: 13000円,敷金: 10.5万円,礼金: 10.5万円,保証金: -,敷引・償却: -,東京都中野区若宮３,西武新宿線/都立家政駅 歩7分,1K,...,2007年11月,要,-,即,仲介,事務所利用相談,2007110931,100177010514,9戸,https://suumo.jp/chintai/jnc_000033610878/?bc=...
1,フュージョナル五反野,6.9万円,管理費・共益費: 12000円,敷金: 6.9万円,礼金: 6.9万円,保証金: -,敷引・償却: -,東京都足立区足立１,東武伊勢崎線/五反野駅 歩10分,1K,...,2019年7月,要,-,'22年11月中旬,仲介,-,R00219-001190,100298995067,41戸,https://suumo.jp/chintai/jnc_000049871528/?bc=...
2,サークルハウス北千住 106号室,4.9万円,管理費・共益費: 3000円,敷金: -,礼金: -,保証金: -,敷引・償却: -,東京都足立区千住仲町,東京メトロ千代田線/北千住駅 歩7分,ワンルーム,...,2016年2月,要,-,即,仲介,-,73731,100299233987,10戸,https://suumo.jp/chintai/jnc_000075164463/?bc=...
3,ストーク森山ビル,16万円,管理費・共益費: -,敷金: 16万円,礼金: 16万円,保証金: -,敷引・償却: -,東京都文京区本郷２,東京メトロ丸ノ内線/本郷三丁目駅 歩4分,2LDK,...,1980年11月,要,近隣54m38500円,即,仲介,二人入居可/子供可,57854344,100299818419,-,https://suumo.jp/chintai/jnc_000060490279/?bc=...
4,京王井の頭線 西永福駅 3階建 築35年,16.8万円,管理費・共益費: 10000円,敷金: 16.8万円,礼金: 16.8万円,保証金: -,敷引・償却: -,東京都杉並区浜田山１,京王井の頭線/西永福駅 歩4分,3LDK,...,1988年6月,要,敷地内24200円,'22年12月中旬,仲介,単身者可/二人入居可/子供可/楽器相談,0000016413,100301272810,-,https://suumo.jp/chintai/jnc_000077560758/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105675,東武伊勢崎線 梅島駅 2階建 築35年,4.5万円,管理費・共益費: 1000円,敷金: -,礼金: -,保証金: -,敷引・償却: -,東京都足立区梅田５,東武伊勢崎線/梅島駅 歩7分,1K,...,1988年2月,要,-,即,代理,子供不可/事務所利用不可/ルームシェア不可,-,100276133359,-,https://suumo.jp/chintai/jnc_000077666436/?bc=...
105676,東京メトロ半蔵門線 住吉駅 3階建 築38年,9万円,管理費・共益費: 3000円,敷金: 9万円,礼金: -,保証金: -,敷引・償却: -,東京都江東区千田,東京メトロ半蔵門線/住吉駅 歩11分,2DK,...,1985年1月,要,-,相談,仲介,-,ty0000041779,100291602676,-,https://suumo.jp/chintai/jnc_000069250547/?bc=...
105677,ルーブル亀戸天神前,8万円,管理費・共益費: 9000円,敷金: -,礼金: 8万円,保証金: -,敷引・償却: -,東京都江東区亀戸３,東京メトロ半蔵門線/錦糸町駅 歩11分,1K,...,2014年4月,2万円2年,-,'22年12月中旬,代理,単身者限定/子供不可/事務所利用不可/ルームシェア不可,128277,100301012977,120戸,https://suumo.jp/chintai/jnc_000077523861/?bc=...
105678,エトワールブランシュ新橋,11.4万円,管理費・共益費: 6000円,敷金: -,礼金: 11.4万円,保証金: -,敷引・償却: -,東京都港区新橋４,ＪＲ山手線/新橋駅 歩6分,1K,...,2014年1月,要,敷地内38500円,'22年11月上旬,仲介,単身者可,HS0563,100294075873,-,https://suumo.jp/chintai/jnc_000076124149/?bc=...


In [507]:
# 変数多すぎません？使わないやつ減らしましょうよ。
colnames = [
    '物件名',
    '家賃',  # 毎月かかるお金。
    '管理費・共益費',  # 物件の維持コスト。共益費は特に建物の共有部分の維持コスト。
    '敷金',  # 滞納とか契約終了時の原状復帰とかに利用される。余ったら返却される。
    '礼金',  # 物件貸し出しに対するお礼。
    '保証金',  # 基本的に敷金と同じ。礼金が含まれている場合もある。
    '敷引・償却',  # 返却されない敷金のこと。
    '住所',
    '駅徒歩',
    '間取り',
    '面積',
    '築年数',
    '階',
    '向き',
    '建物種別',
    'オプション',
    '間取り詳細',
    '構造',  # 建物の材質（木造とか）
    '階建',
    '築年月',  # 築年数で良くない?
    '損保',  # 不要?
    '駐車場',
    '入居',  # すぐ入居できますか？即 or 相談 or ??年??月?旬
    '取引態様',  # '貸主', '仲介', '代理'
    '条件',
    '取り扱い店舗物件コード',  # 不要
    'SUUMO物件コード',  # 実質ユニークキー
    '総戸数',  # 建物内の部屋の数
    'url'
]

In [18]:
suumo_details = pd.read_csv('./data/csv/SuumoDetails20221031.csv')
suumo_details

,物件名,家賃,管理費・共益費,敷金,礼金,保証金,敷引・償却,住所,駅徒歩,間取り,...,築年月,損保,駐車場,入居,取引態様,条件,取り扱い店舗物件コード,SUUMO物件コード,総戸数,url
0,プチ・ボヌール,14.8,5000,14.8,14.8,0.0,0,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,42DK,...,2019年2月,2.4万円2年,-,即,貸主,-,-,100294945613,-,https://suumo.jp/chintai/jnc_000046696863/?bc=...
1,ポルトボヌール,11.5,3000,0.0,11.5,0.0,0,東京都荒川区東尾久６,東京メトロ千代田線/町屋駅 歩14分,2LDK,...,2017年1月,要,-,'22年12月中旬,仲介,二人入居可,R01278-012075,100302064363,-,https://suumo.jp/chintai/jnc_000077689180/?bc=...
2,日暮里・舎人ライナー 熊野前駅 3階建 築6年,11.5,3000,0.0,11.5,0.0,0,東京都荒川区東尾久６,日暮里・舎人ライナー/熊野前駅 歩7分,2LDK,...,2017年1月,要,-,'22年12月中旬,仲介,-,306-36865,100302075201,-,https://suumo.jp/chintai/jnc_000077692991/?bc=...
3,Ｃｒｅｓｔ Ｌｅ Ｃｅｉｌ,14.4,8000,14.4,14.4,0.0,0,東京都荒川区西尾久２,都電荒川線/宮ノ前駅 歩5分,2LDK,...,2020年11月,要,-,'23年1月下旬,仲介,二人入居可/子供可/ルームシェア不可,3320460,100302058292,7戸,https://suumo.jp/chintai/jnc_000077702975/?bc=...
4,Ｃｒｅｓｔ Ｌｅ Ｃｅｉｌ,14.5,8000,14.5,14.5,0.0,0,東京都荒川区西尾久２,都電荒川線/宮ノ前駅 歩5分,2LDK,...,2020年11月,要,-,'22年12月上旬,仲介,二人入居可/子供可/ルームシェア不可,3320461,100302058345,7戸,https://suumo.jp/chintai/jnc_000077702976/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90408,ＰＥＡＲＬハイツ 101号室,5.8,0,0.0,0.0,0.0,0,東京都足立区島根１,東武伊勢崎線/梅島駅 歩10分,1LDK,...,1989年3月,要,近隣300m13000円,即,仲介,二人入居可/ルームシェア不可,00110720-101,100283836059,14戸,https://suumo.jp/chintai/jnc_000075777447/?bc=...
90409,都営大江戸線 六本木駅 4階建 築11年,15.5,10000,15.5,15.5,0.0,0,東京都港区元麻布３,都営大江戸線/六本木駅 歩8分,1K,...,2012年2月,1.8万円2年,近隣50m40000円,即,仲介,-,195722,100293511010,-,https://suumo.jp/chintai/jnc_000076195400/?bc=...
90410,都営大江戸線 六本木駅 4階建 築11年,27.5,10000,27.5,27.5,0.0,0,東京都港区元麻布３,都営大江戸線/六本木駅 歩8分,3LDK,...,2012年2月,1.8万円2年,近隣50m40000円,即,仲介,二人入居可/子供可,198114,100295378533,-,https://suumo.jp/chintai/jnc_000059246188/?bc=...
90411,ベルエア野方,6.4,3000,3.2,3.2,0.0,0,東京都中野区野方４,西武新宿線/野方駅 歩6分,ワンルーム,...,2003年9月,要,近隣531m16500円,即,仲介,子供不可,57462387,100294439475,-,https://suumo.jp/chintai/jnc_000076359710/?bc=...


In [12]:
# ???万円→???
def drop_manyen(x):
    return re.sub(r'([0-9]+)万円',r'\1',x)

# ???円→???
def drop_yen(x):
    return re.sub(r'([0-9]+)円',r'\1',x)

# 文字+数字+文字 → 数字
def objnumobj_num(x):
    return re.sub(r'\w+?([0-9]+)\w+',r'\1',x)

# 最後の数字列を取り出す
def lastnum(x):
    return re.findall('[0-9]+',x)[-1]

# (文字)数字を取り出す
def get_objnum(x):
    return re.search(r'([A-Z]?)([0-9]+)',x).group()

# 住所→緯度経度変換装置
import requests
import urllib

def translate_address_coordinates(address):
    makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
    s_quote = urllib.parse.quote(address)
    response = requests.get(makeUrl + s_quote)
    return response.json()[0]["geometry"]["coordinates"]

# 座標から距離を計算
def distance(keido1,ido1,keido2,ido2):
    import math
    # 出力は[m]
    r=6378137
    keido1 = math.radians(keido1)
    ido1 = math.radians(ido1)
    keido2 = math.radians(keido2)
    ido2 = math.radians(ido2)
    distance = r*np.arccos(round(np.sin(ido1)*np.sin(ido2)+np.cos(ido1)*np.cos(ido2)*np.cos(keido2-keido1),10))
    return distance

In [17]:
# 家賃変数

# ??.?万円　→　??.?(float)
suumo_details['家賃'] = suumo_details['家賃'].map(lambda x:drop_manyen(x)).astype(float)

TypeError: expected string or bytes-like object

In [617]:
# 管理費・共益費変数

# (管理費・共益費)を除去
def process_kannrihi_Remove_unnecessary_characters(x):
    return re.search('(管理費・共益費:\xa0)(.+)',x).group(2)
suumo_details['管理費・共益費'] = suumo_details['管理費・共益費'].map(lambda x: process_kannrihi_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['管理費・共益費'] = suumo_details['管理費・共益費'].replace('-','0円')

# 円を除去
suumo_details['管理費・共益費'] = suumo_details['管理費・共益費'].map(lambda x: drop_yen(x))

# int型にする
suumo_details['管理費・共益費'] = suumo_details['管理費・共益費'].astype(int)

In [618]:
# 敷金変数

# (敷金)を除去
def process_shikikin_Remove_unnecessary_characters(x):
    return re.search('(敷金:\xa0)(.+)',x).group(2)
suumo_details['敷金'] = suumo_details['敷金'].map(lambda x: process_shikikin_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['敷金'] = suumo_details['敷金'].replace('-','0万円')

# 万円を除去
suumo_details['敷金'] = suumo_details['敷金'].map(lambda x: drop_manyen(x))

# float型にする
suumo_details['敷金'] = suumo_details['敷金'].astype(float)

In [619]:
# 礼金変数

# (礼金)を除去
def process_reikin_Remove_unnecessary_characters(x):
    return re.search('(礼金:\xa0)(.+)',x).group(2)
suumo_details['礼金'] = suumo_details['礼金'].map(lambda x: process_reikin_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['礼金'] = suumo_details['礼金'].replace('-','0万円')

# 万円を除去
suumo_details['礼金'] = suumo_details['礼金'].map(lambda x: drop_manyen(x))

# float型にする
suumo_details['礼金'] = suumo_details['礼金'].astype(float)

In [620]:
# 保証金変数

# (保証金)を除去
def process_hosyokin_Remove_unnecessary_characters(x):
    return re.search('(保証金:\xa0)(.+)',x).group(2)
suumo_details['保証金'] = suumo_details['保証金'].map(lambda x: process_hosyokin_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['保証金'] = suumo_details['保証金'].replace('-','0万円')

# 万円を除去
suumo_details['保証金'] = suumo_details['保証金'].map(lambda x: drop_manyen(x))

# float型にする
suumo_details['保証金'] = suumo_details['保証金'].astype(float)

In [18]:
# 敷引・償却変数
# 実費ってなんすか…？

# (敷引・償却)を除去
def process_shikihiki_Remove_unnecessary_characters(x):
    return re.search('(敷引・償却:\xa0)(.+)',x).group(2)
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].map(lambda x: process_shikihiki_Remove_unnecessary_characters(x))

# 「-」「実費」は0円ということにする
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].replace('-','0万円')
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].replace('実費','0万円')

# 万円を除去
consition = suumo_details['敷引・償却'].map
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].map(lambda x: drop_manyen(x))

# float型にする
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].astype(float)

TypeError: expected string or bytes-like object

In [19]:
# 住所変数

# ～都,～区,～ に分けたい
# ついでに[都道府県][市区町村][以下]に対応したい

# 住所を(都道府県)(市区町村)(市区町村以下)に分けて取り出す
def split_address(x):
    a,b,c = re.search('(...??[都道府県])(.+?[市区町村])(.+)',x).groups()
    return a,b,c

# それぞれを変数に入れる
suumo_details[['都道府県','市区町村','市区町村以下']] = suumo_details.apply(lambda x:split_address(x.住所),axis=1,result_type='expand')

In [20]:
# 住所のuniqueから緯度経度を取得
# csvに変換しておいたので2回目は不要　./data/csv/coordinates_details.csvを呼び出そう

all_addresses = {}
running_times = []
i = 0

for address in suumo_details['住所'].unique():
    
    start = time.time()

    coordinate = translate_address_coordinates(address)
    all_addresses[address] = coordinate
    time.sleep(1)
    
    finish = time.time()
    
    running_time = finish - start
    running_times.append(running_time)

    all_count = suumo_details['住所'].unique().shape[0]
    print(f'{i}件目：{running_time}秒')
    # 作業進捗
    complete_ratio = round(i/all_count*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(running_times)
    running_required_time = running_mean * (all_count - i)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')
    i += 1

0件目：1.1348462104797363秒
完了：0.0%
残り時間：0時間51分4秒

1件目：1.155693769454956秒
完了：0.037%
残り時間：0時間51分31秒

2件目：1.0982301235198975秒
完了：0.074%
残り時間：0時間50分47秒

3件目：1.121762990951538秒
完了：0.111%
残り時間：0時間50分41秒

4件目：1.112973928451538秒
完了：0.148%
残り時間：0時間50分32秒

5件目：1.1557931900024414秒
完了：0.185%
残り時間：0時間50分45秒

6件目：1.1478421688079834秒
完了：0.222%
残り時間：0時間50分50秒

7件目：1.0943129062652588秒
完了：0.259%
残り時間：0時間50分36秒

8件目：1.172168493270874秒
完了：0.296%
残り時間：0時間50分49秒

9件目：1.1315717697143555秒
完了：0.333%
残り時間：0時間50分47秒

10件目：1.1449215412139893秒
完了：0.37%
残り時間：0時間50分49秒

11件目：1.1338169574737549秒
完了：0.407%
残り時間：0時間50分48秒

12件目：1.1207818984985352秒
完了：0.444%
残り時間：0時間50分44秒

13件目：1.1280694007873535秒
完了：0.481%
残り時間：0時間50分42秒

14件目：1.1247203350067139秒
完了：0.519%
残り時間：0時間50分40秒

15件目：1.1266615390777588秒
完了：0.556%
残り時間：0時間50分38秒

16件目：1.1462078094482422秒
完了：0.593%
残り時間：0時間50分39秒

17件目：1.11268949508667秒
完了：0.63%
残り時間：0時間50分35秒

18件目：1.1155428886413574秒
完了：0.667%
残り時間：0時間50分31秒

19件目：1.136897087097168秒
完了：0.704%
残り時間：0時間50分31秒

20

162件目：1.144373893737793秒
完了：6.0%
残り時間：0時間48分1秒

163件目：1.1358840465545654秒
完了：6.037%
残り時間：0時間48分0秒

164件目：1.141411542892456秒
完了：6.074%
残り時間：0時間47分59秒

165件目：1.14347243309021秒
完了：6.111%
残り時間：0時間47分58秒

166件目：1.1352715492248535秒
完了：6.148%
残り時間：0時間47分57秒

167件目：1.128572702407837秒
完了：6.185%
残り時間：0時間47分55秒

168件目：1.1282389163970947秒
完了：6.222%
残り時間：0時間47分54秒

169件目：1.1050002574920654秒
完了：6.259%
残り時間：0時間47分53秒

170件目：1.1167864799499512秒
完了：6.296%
残り時間：0時間47分51秒

171件目：1.155179738998413秒
完了：6.333%
残り時間：0時間47分50秒

172件目：1.1200940608978271秒
完了：6.37%
残り時間：0時間47分49秒

173件目：1.1312155723571777秒
完了：6.407%
残り時間：0時間47分48秒

174件目：1.1497688293457031秒
完了：6.444%
残り時間：0時間47分47秒

175件目：1.1350772380828857秒
完了：6.481%
残り時間：0時間47分46秒

176件目：1.1451315879821777秒
完了：6.519%
残り時間：0時間47分45秒

177件目：1.131281852722168秒
完了：6.556%
残り時間：0時間47分43秒

178件目：1.1442341804504395秒
完了：6.593%
残り時間：0時間47分42秒

179件目：1.1345858573913574秒
完了：6.63%
残り時間：0時間47分41秒

180件目：1.1338250637054443秒
完了：6.667%
残り時間：0時間47分40秒

181件目：1.1335039138793945秒

321件目：1.1577560901641846秒
完了：11.889%
残り時間：0時間45分11秒

322件目：1.1260087490081787秒
完了：11.926%
残り時間：0時間45分10秒

323件目：1.1317977905273438秒
完了：11.963%
残り時間：0時間45分9秒

324件目：1.1605453491210938秒
完了：12.0%
残り時間：0時間45分8秒

325件目：1.1197764873504639秒
完了：12.037%
残り時間：0時間45分7秒

326件目：1.1674461364746094秒
完了：12.074%
残り時間：0時間45分6秒

327件目：1.1302950382232666秒
完了：12.111%
残り時間：0時間45分5秒

328件目：1.1626853942871094秒
完了：12.148%
残り時間：0時間45分4秒

329件目：1.1571903228759766秒
完了：12.185%
残り時間：0時間45分3秒

330件目：1.1158950328826904秒
完了：12.222%
残り時間：0時間45分1秒

331件目：1.1539719104766846秒
完了：12.259%
残り時間：0時間45分0秒

332件目：1.1060404777526855秒
完了：12.296%
残り時間：0時間44分59秒

333件目：1.1342487335205078秒
完了：12.333%
残り時間：0時間44分58秒

334件目：1.1649091243743896秒
完了：12.37%
残り時間：0時間44分57秒

335件目：1.1647114753723145秒
完了：12.407%
残り時間：0時間44分56秒

336件目：1.180426836013794秒
完了：12.444%
残り時間：0時間44分55秒

337件目：1.1647167205810547秒
完了：12.481%
残り時間：0時間44分54秒

338件目：1.1482584476470947秒
完了：12.519%
残り時間：0時間44分53秒

339件目：1.1478033065795898秒
完了：12.556%
残り時間：0時間44分52秒

340件目：

478件目：1.1297814846038818秒
完了：17.704%
残り時間：0時間42分19秒

479件目：1.1348695755004883秒
完了：17.741%
残り時間：0時間42分18秒

480件目：1.1202478408813477秒
完了：17.778%
残り時間：0時間42分17秒

481件目：1.102116346359253秒
完了：17.815%
残り時間：0時間42分15秒

482件目：1.1471059322357178秒
完了：17.852%
残り時間：0時間42分14秒

483件目：1.2224695682525635秒
完了：17.889%
残り時間：0時間42分13秒

484件目：1.1331207752227783秒
完了：17.926%
残り時間：0時間42分12秒

485件目：1.1668331623077393秒
完了：17.963%
残り時間：0時間42分11秒

486件目：1.1415743827819824秒
完了：18.0%
残り時間：0時間42分10秒

487件目：1.1410388946533203秒
完了：18.037%
残り時間：0時間42分9秒

488件目：1.1634535789489746秒
完了：18.074%
残り時間：0時間42分8秒

489件目：1.1625354290008545秒
完了：18.111%
残り時間：0時間42分7秒

490件目：1.1581952571868896秒
完了：18.148%
残り時間：0時間42分6秒

491件目：1.132817029953003秒
完了：18.185%
残り時間：0時間42分4秒

492件目：1.1223020553588867秒
完了：18.222%
残り時間：0時間42分3秒

493件目：1.179905891418457秒
完了：18.259%
残り時間：0時間42分2秒

494件目：1.1220173835754395秒
完了：18.296%
残り時間：0時間42分1秒

495件目：1.114027976989746秒
完了：18.333%
残り時間：0時間42分0秒

496件目：1.1123173236846924秒
完了：18.37%
残り時間：0時間41分58秒

497件目：1.1

635件目：1.1780598163604736秒
完了：23.519%
残り時間：0時間39分22秒

636件目：1.1476457118988037秒
完了：23.556%
残り時間：0時間39分21秒

637件目：1.096181869506836秒
完了：23.593%
残り時間：0時間39分20秒

638件目：1.154951572418213秒
完了：23.63%
残り時間：0時間39分19秒

639件目：1.1623923778533936秒
完了：23.667%
残り時間：0時間39分18秒

640件目：1.1279230117797852秒
完了：23.704%
残り時間：0時間39分17秒

641件目：1.1284713745117188秒
完了：23.741%
残り時間：0時間39分15秒

642件目：1.1218364238739014秒
完了：23.778%
残り時間：0時間39分14秒

643件目：1.0989809036254883秒
完了：23.815%
残り時間：0時間39分13秒

644件目：1.0947716236114502秒
完了：23.852%
残り時間：0時間39分12秒

645件目：1.134037733078003秒
完了：23.889%
残り時間：0時間39分10秒

646件目：1.1177830696105957秒
完了：23.926%
残り時間：0時間39分9秒

647件目：1.109508752822876秒
完了：23.963%
残り時間：0時間39分8秒

648件目：1.1222529411315918秒
完了：24.0%
残り時間：0時間39分7秒

649件目：1.1197631359100342秒
完了：24.037%
残り時間：0時間39分6秒

650件目：1.115624189376831秒
完了：24.074%
残り時間：0時間39分4秒

651件目：1.1273748874664307秒
完了：24.111%
残り時間：0時間39分3秒

652件目：1.1330595016479492秒
完了：24.148%
残り時間：0時間39分2秒

653件目：1.135251522064209秒
完了：24.185%
残り時間：0時間39分1秒

654件目：1.15

...

2028件目：1.1459505558013916秒
完了：75.111%
残り時間：0時間12分55秒

2029件目：1.0994718074798584秒
完了：75.148%
残り時間：0時間12分54秒

2030件目：1.1355664730072021秒
完了：75.185%
残り時間：0時間12分53秒

2031件目：1.1424694061279297秒
完了：75.222%
残り時間：0時間12分52秒

2032件目：1.1516392230987549秒
完了：75.259%
残り時間：0時間12分51秒

2033件目：1.1301934719085693秒
完了：75.296%
残り時間：0時間12分49秒

2034件目：1.0996942520141602秒
完了：75.333%
残り時間：0時間12分48秒

2035件目：1.1319589614868164秒
完了：75.37%
残り時間：0時間12分47秒

2036件目：1.1262030601501465秒
完了：75.407%
残り時間：0時間12分46秒

2037件目：1.107229471206665秒
完了：75.444%
残り時間：0時間12分45秒

2038件目：1.1142983436584473秒
完了：75.481%
残り時間：0時間12分44秒

2039件目：1.1009838581085205秒
完了：75.519%
残り時間：0時間12分42秒

2040件目：1.0986354351043701秒
完了：75.556%
残り時間：0時間12分41秒

2041件目：1.1386761665344238秒
完了：75.593%
残り時間：0時間12分40秒

2042件目：1.1011977195739746秒
完了：75.63%
残り時間：0時間12分39秒

2043件目：1.111609697341919秒
完了：75.667%
残り時間：0時間12分38秒

2044件目：1.1115262508392334秒
完了：75.704%
残り時間：0時間12分37秒

2045件目：1.1283652782440186秒
完了：75.741%
残り時間：0時間12分35秒

2046件目：1.1456594467163086秒
完了：75

2182件目：1.124587059020996秒
完了：80.815%
残り時間：0時間9分57秒

2183件目：1.1445057392120361秒
完了：80.852%
残り時間：0時間9分56秒

2184件目：1.1465682983398438秒
完了：80.889%
残り時間：0時間9分55秒

2185件目：1.2245829105377197秒
完了：80.926%
残り時間：0時間9分54秒

2186件目：1.1536450386047363秒
完了：80.963%
残り時間：0時間9分52秒

2187件目：1.1287195682525635秒
完了：81.0%
残り時間：0時間9分51秒

2188件目：1.1359829902648926秒
完了：81.037%
残り時間：0時間9分50秒

2189件目：1.1508233547210693秒
完了：81.074%
残り時間：0時間9分49秒

2190件目：1.1357955932617188秒
完了：81.111%
残り時間：0時間9分48秒

2191件目：1.1187334060668945秒
完了：81.148%
残り時間：0時間9分47秒

2192件目：1.1173837184906006秒
完了：81.185%
残り時間：0時間9分45秒

2193件目：1.1030056476593018秒
完了：81.222%
残り時間：0時間9分44秒

2194件目：1.104593276977539秒
完了：81.259%
残り時間：0時間9分43秒

2195件目：1.1258525848388672秒
完了：81.296%
残り時間：0時間9分42秒

2196件目：1.1308612823486328秒
完了：81.333%
残り時間：0時間9分41秒

2197件目：1.1147804260253906秒
完了：81.37%
残り時間：0時間9分40秒

2198件目：1.101179838180542秒
完了：81.407%
残り時間：0時間9分38秒

2199件目：1.191002368927002秒
完了：81.444%
残り時間：0時間9分37秒

2200件目：1.143221378326416秒
完了：81.481%
残り時間：0時間9分36秒

2

2339件目：1.1100008487701416秒
完了：86.63%
残り時間：0時間6分55秒

2340件目：1.1396663188934326秒
完了：86.667%
残り時間：0時間6分54秒

2341件目：1.1187002658843994秒
完了：86.704%
残り時間：0時間6分53秒

2342件目：1.1065475940704346秒
完了：86.741%
残り時間：0時間6分52秒

2343件目：1.139282464981079秒
完了：86.778%
残り時間：0時間6分51秒

2344件目：1.1016135215759277秒
完了：86.815%
残り時間：0時間6分50秒

2345件目：1.1109647750854492秒
完了：86.852%
残り時間：0時間6分49秒

2346件目：1.1048986911773682秒
完了：86.889%
残り時間：0時間6分47秒

2347件目：1.1433742046356201秒
完了：86.926%
残り時間：0時間6分46秒

2348件目：1.1054916381835938秒
完了：86.963%
残り時間：0時間6分45秒

2349件目：1.1254665851593018秒
完了：87.0%
残り時間：0時間6分44秒

2350件目：1.1340200901031494秒
完了：87.037%
残り時間：0時間6分43秒

2351件目：1.1124567985534668秒
完了：87.074%
残り時間：0時間6分42秒

2352件目：1.1306209564208984秒
完了：87.111%
残り時間：0時間6分40秒

2353件目：1.1314311027526855秒
完了：87.148%
残り時間：0時間6分39秒

2354件目：1.1439528465270996秒
完了：87.185%
残り時間：0時間6分38秒

2355件目：1.1221275329589844秒
完了：87.222%
残り時間：0時間6分37秒

2356件目：1.1306607723236084秒
完了：87.259%
残り時間：0時間6分36秒

2357件目：1.1169779300689697秒
完了：87.296%
残り時間：0時間6分35

2496件目：1.1077589988708496秒
完了：92.444%
残り時間：0時間3分54秒

2497件目：1.1239964962005615秒
完了：92.481%
残り時間：0時間3分53秒

2498件目：1.107001543045044秒
完了：92.519%
残り時間：0時間3分52秒

2499件目：1.1097829341888428秒
完了：92.556%
残り時間：0時間3分51秒

2500件目：1.119999885559082秒
完了：92.593%
残り時間：0時間3分50秒

2501件目：1.1228086948394775秒
完了：92.63%
残り時間：0時間3分48秒

2502件目：1.1259479522705078秒
完了：92.667%
残り時間：0時間3分47秒

2503件目：1.1207013130187988秒
完了：92.704%
残り時間：0時間3分46秒

2504件目：1.119795560836792秒
完了：92.741%
残り時間：0時間3分45秒

2505件目：1.1192419528961182秒
完了：92.778%
残り時間：0時間3分44秒

2506件目：1.1498403549194336秒
完了：92.815%
残り時間：0時間3分43秒

2507件目：1.1202971935272217秒
完了：92.852%
残り時間：0時間3分42秒

2508件目：1.1261329650878906秒
完了：92.889%
残り時間：0時間3分40秒

2509件目：1.133129596710205秒
完了：92.926%
残り時間：0時間3分39秒

2510件目：1.1119742393493652秒
完了：92.963%
残り時間：0時間3分38秒

2511件目：1.1171953678131104秒
完了：93.0%
残り時間：0時間3分37秒

2512件目：1.1539158821105957秒
完了：93.037%
残り時間：0時間3分36秒

2513件目：1.1210641860961914秒
完了：93.074%
残り時間：0時間3分35秒

2514件目：1.1019268035888672秒
完了：93.111%
残り時間：0時間3分33秒



2653件目：1.1353216171264648秒
完了：98.259%
残り時間：0時間0分54秒

2654件目：1.1129951477050781秒
完了：98.296%
残り時間：0時間0分52秒

2655件目：1.2392363548278809秒
完了：98.333%
残り時間：0時間0分51秒

2656件目：1.344529151916504秒
完了：98.37%
残り時間：0時間0分50秒

2657件目：1.1411588191986084秒
完了：98.407%
残り時間：0時間0分49秒

2658件目：1.1521317958831787秒
完了：98.444%
残り時間：0時間0分48秒

2659件目：1.1011719703674316秒
完了：98.481%
残り時間：0時間0分47秒

2660件目：1.1050360202789307秒
完了：98.519%
残り時間：0時間0分46秒

2661件目：1.099029541015625秒
完了：98.556%
残り時間：0時間0分44秒

2662件目：1.1111814975738525秒
完了：98.593%
残り時間：0時間0分43秒

2663件目：1.1518816947937012秒
完了：98.63%
残り時間：0時間0分42秒

2664件目：1.137476921081543秒
完了：98.667%
残り時間：0時間0分41秒

2665件目：1.1167612075805664秒
完了：98.704%
残り時間：0時間0分40秒

2666件目：1.1302661895751953秒
完了：98.741%
残り時間：0時間0分39秒

2667件目：1.122035026550293秒
完了：98.778%
残り時間：0時間0分37秒

2668件目：1.1505043506622314秒
完了：98.815%
残り時間：0時間0分36秒

2669件目：1.1101365089416504秒
完了：98.852%
残り時間：0時間0分35秒

2670件目：1.1141746044158936秒
完了：98.889%
残り時間：0時間0分34秒

2671件目：1.1059620380401611秒
完了：98.926%
残り時間：0時間0分33秒


In [21]:
# 住所と緯度経度のDataFrame
# これもcsvに保存済なので2回目は不要

coordinates = pd.DataFrame(all_addresses).T
coordinates.reset_index(inplace=True)
coordinates.rename(columns={
    'index':'住所',
    0:'経度',
    1:'緯度'
}, inplace=True)
coordinates.to_csv('./data/csv/coordinates_details2.csv',index=False)

In [22]:
# 元データとくっつける
# 連打すると増殖するので注意
coordinates_details = pd.read_csv('./data/csv/coordinates_details2.csv')
suumo_details = pd.merge(suumo_details, coordinates_details, on='住所', how='left')

In [25]:
# 最寄り駅変数（正規表現）

# 最寄り駅から路線と駅を取り出す
suumo_details[['路線','駅']] = suumo_details.apply(lambda x:re.search('(.+)/(.+?)\s(.+)',x.駅徒歩).groups()[0:2],axis=1,result_type='expand')

def how_to_station():
    # 移動手段は3パターン
    how_to_station = ['歩','バス','車']
    for by in how_to_station:
        # 移動手段変数を生成
        suumo_details[f'{by}'] = np.zeros(suumo_details.shape[0])
        # その移動手段があるなら移動手段変数に格納する
        condition = suumo_details['駅徒歩'].map(lambda x: search_object(f'{by}[0-9]+分',x))
        suumo_details.loc[condition,f'{by}'] = suumo_details.loc[condition,'駅徒歩'].map(lambda x: re.search(f'{by}([0-9]+)分',x).group(1))
        # 所要時間をint型に変換
        suumo_details[f'{by}'] = suumo_details[f'{by}'].astype(int)

# 移動手段ごとに所用時間変数を作る
how_to_station()

In [26]:
# 駅の緯度経度情報を結合

# 駅情報を読み込み
station_coordinates = pd.read_csv('./data/csv/station_coordinates.csv')
# 名前が被らないように修正
station_coordinates.rename(columns={
    '住所':'住所(駅)',
    '緯度':'緯度(駅)',
    '経度':'経度(駅)'
}, inplace=True)
# 〇〇駅に名称を変換
station_coordinates['駅'] = station_coordinates['駅'].map(lambda x: x+'駅')

# ヶはケに変える
condition = suumo_details['駅'].map(lambda x: search_object('ヶ',x))
suumo_details.loc[condition,'駅'] = suumo_details.loc[condition,'駅'].map(lambda x: re.sub('ヶ','ケ',x))
# 結合
suumo_details = pd.merge(suumo_details, station_coordinates, on='駅', how='left')

In [27]:
# 最寄駅からの距離変数を生成

# 最寄駅からの距離[m]
suumo_details['最寄駅からの距離'] = 0
condition = ~suumo_details['経度(駅)'].isnull()
suumo_details.loc[condition,'最寄駅からの距離'] = suumo_details[condition].apply(lambda x: distance(x['経度'], x['緯度'], x['経度(駅)'], x['緯度(駅)']), axis=1)

In [28]:
# 皇居からの距離変数を生成

# 皇居の座標を獲得
koukyo = '東京都千代田区千代田1'
keido_k, ido_k = translate_address_coordinates(koukyo)

# 皇居(都心？)からの距離[m]
suumo_details['皇居からの距離'] = 0
suumo_details['皇居からの距離'] = suumo_details.apply(lambda x: distance(x['経度'], x['緯度'], keido_k, ido_k), axis=1)

In [29]:
# 間取り変数の変換

# ワンルームは1部屋ってことにする（かなり諸説）
condition = suumo_details['間取り'] == 'ワンルーム'
suumo_details.loc[condition,'間取り'] = '1'

# 数字部分を取り出して、部屋数変数にする
suumo_details['部屋数'] = suumo_details['間取り'].map(lambda x: re.search('([0-9]+)(.*)',x).group(1)).astype(int)

# S,L,D,K はあれば1でなければ0になる
list_alpha = ['S','L','D','K']
for alpha in list_alpha:
    suumo_details[f'間取り_{alpha}'] = 0
    condition = suumo_details['間取り'].map(lambda x: search_object(alpha,x))
    suumo_details.loc[condition,f'間取り_{alpha}'] = 1

In [30]:
# 面積変数

# ??.??m2 → ??.??
def process_area(x):
    return re.search('(.+)(m2)',x).group(1)
suumo_details['面積'] = suumo_details['面積'].map(lambda x: process_area(x))

# float型に変換
suumo_details['面積'] = suumo_details['面積'].astype('float')

In [31]:
# 築年数変数

# 新築は築0年ということにする
suumo_details.loc[suumo_details['築年数']=='新築','築年数'] = '築0年'

# 築??年(以上)→??
# 99階以上は99階に圧縮されます
suumo_details['築年数'] = suumo_details['築年数'].map(lambda x: objnumobj_num(x))

# 整数型に変更
suumo_details['築年数'] = suumo_details['築年数'].astype(int)

In [32]:
# 階変数

# - は1階ということにする
suumo_details.loc[suumo_details['階']=='-','階'] = '1階'

# 階の前処理 -があればその後ろ、なければ(文字)数字
def floor(x):
    return search_object('-',x)*lastnum(x)+(1-search_object('-',x))*get_objnum(x)
# B? → -?,Bがなければそのまま
def basement_floor(x):
    return search_object('B',x)*('-'+re.search(r'[0-9]+',x).group())+(1-search_object('B',x))*(x)
# M? → ?.5 Mがなければそのまま
def middle_floor(x):
    return search_object('M',x)*str((int(re.search(r'[0-9]+',x).group())-0.5))+(1-search_object('M',x))*(x)
   
# 上の処理を適用する
suumo_details['階'] = suumo_details['階'].map(lambda x: floor(x)).map(lambda x: basement_floor(x)).map(lambda x: middle_floor(x))

# float型に変換
suumo_details['階'] = suumo_details['階'].astype(float)

In [33]:
# 向き変数は処理不要
# 「-」もそのままにする：そこそこ数がある＋正解らしい処理も見当たらない

In [34]:
# 建物種別変数は処理不要

In [35]:
# 間取り詳細は不採用
# 処理難度が高い気がする

In [36]:
# 構造変数は処理不要

In [37]:
# 階建変数→階数変数
# 「/」で分けられていない皆さんがいますがそのまま階数変数に入れます。
condition = suumo_details['階建'].map(lambda x: search_object('/',x))
suumo_details.loc[condition,'階数'] = suumo_details.loc[condition,'階建'].map(lambda x: re.search('(.+)/(.+)',x).group(2))
suumo_details.loc[~condition,'階数'] = suumo_details.loc[~condition,'階建']

In [38]:
# 地下変数

# 平屋は1階建ということにする
suumo_details.loc[suumo_details['階数']=='平屋','階数'] = '1階建'

# 地下何階までありますか？関数
def underground(x):
    return search_object('地下',x)*objnumobj_num(x)+(1-search_object('地下',x))*'0'
    
# 地下がなければ０、あるなら何階まであるか
suumo_details['地下'] = suumo_details['階数'].map(lambda x: underground(x))

# int型に変換
suumo_details['地下'] = suumo_details['地下'].astype('int')

In [39]:
# 地上変数(正規表現)

# 平屋は1階建ということにする
suumo_details.loc[suumo_details['階数']=='平屋','階数'] = '1階建'

# ??階建→??
suumo_details['地上'] = suumo_details['階数'].map(lambda x: lastnum(x))

# int型に変換
suumo_details['地上'] = suumo_details['地上'].astype(int)

In [40]:
# 階数変数は地下地上の合算にしておく
suumo_details['階数'] = suumo_details['地上'] + suumo_details['地下']

In [41]:
# 築年月変数いる？築年数変数で良くね？

In [42]:
# 損保は…まあええか

In [43]:
# 駐車場変数
# 機械式駐ってのはタワーパーキングのこと
# 駐車場あるorない + 駐車場までの距離 + 駐車場の料金 でどう？
# オプション情報はなくなります…
# 近隣 → 1 + ??[m] + ??[円]
# 敷地内 → 1 + 0[m] + ??[円]
# - → 0 + 0 + 0
# その他　→ 1 + 0[m] + 0[円]
# 問題点は、駐車場がない皆さんの0[m]や0[円]は家賃安いはずなんだけど
# 駐車場がある皆さんは0[m]や0[円]が高いはずってこと

# いったん駐車場がないってことにする
suumo_details[['駐車場有無','駐車場距離','駐車場料金']] = 0,0,0

# 「-」じゃない皆さんには駐車場が与えられます。
condition = suumo_details['駐車場'] == '-'
suumo_details.loc[~condition,'駐車場有無'] = 1

# 近隣の皆さん
condition = suumo_details['駐車場'].map(lambda x: search_object('近隣',x))
# 近隣の皆さんの駐車場距離
def difference_parking(x):
    return re.search('.+?([0-9]+)m([0-9]+)円',x).group(1)
suumo_details.loc[condition,'駐車場距離'] = suumo_details.loc[condition,'駐車場'].map(lambda x: difference_parking(x))
# 近隣の皆さんの駐車場料金
def difference_cost_near_house(x):
    return re.search('.+?([0-9]+)m([0-9]+)円',x).group(2)
suumo_details.loc[condition,'駐車場料金'] = suumo_details.loc[condition,'駐車場'].map(lambda x: difference_cost_near_house(x))

# 敷地内の皆さん
condition = suumo_details['駐車場'].map(lambda x: search_object('敷地内',x))
# 敷地内の皆さんは駐車場料金が発生しません
# 敷地内の皆さんの駐車場料金
def difference_cost_in_house(x):
    return re.search('.+?([0-9]+)円',x).group(1)
suumo_details.loc[condition,'駐車場料金'] = suumo_details.loc[condition,'駐車場'].map(lambda x: difference_cost_in_house(x))

# データ型の変換
suumo_details['駐車場距離'] = suumo_details['駐車場距離'].astype(int)
suumo_details['駐車場料金'] = suumo_details['駐車場料金'].astype(int)

In [44]:
# 入居変数
# ??年??月??旬 → 待ち に変えるよ
soku = suumo_details['入居'] == '即'
soudan = suumo_details['入居'] == '相談'
suumo_details.loc[(~soku)&(~soudan),'入居'] = '待ち'

In [45]:
# 取引態様は処理不要

In [46]:
# 条件変数
# '二人入居可','子供可','ルームシェア不可','単身者可','ペット相談','ルームシェア相談','事務所利用不可','単身者限定','子供不可','楽器相談','フリーレント1ヶ月 (条件),'学生希望','事務所利用相談',
# 使うかどうかは諸説。使うなら使うで取り出しは簡単
# フリーレントとは：入居開始から数カ月家賃無料になる。だいたい条件がある。

In [428]:
# 条件変数の中身
condition_list = []
for i in range(suumo_details.shape[0]):
    condition_list_room = suumo_details.loc[i,'条件'].split('/')
    for cond in condition_list_room:
        condition_list.append(cond)
pd.DataFrame(condition_list)[0].unique().tolist()

['-',
 '二人入居可',
 '子供可',
 'ルームシェア不可',
 '単身者可',
 'ペット相談',
 'ルームシェア相談',
 '事務所利用不可',
 '単身者限定',
 '子供不可',
 '楽器相談',
 'フリーレント1ヶ月 10月契約開始まで',
 '学生希望',
 '事務所利用相談',
 'フリーレント1ヶ月',
 'フリーレント1ヶ月 家賃無料',
 'フリーレント1ヶ月 短期解約違約金あり',
 'フリーレント2ヶ月 ※詳細はお問い合わせくださいませ',
 'フリーレント2ヶ月',
 'フリーレント2ヶ月 ※1年以内解約時賃料等2か月分違約金有',
 'フリーレント2ヶ月 1年以内解約時賃料等2か月分違約金有',
 'フリーレント1ヶ月 2022年12月までの申込',
 'フリーレント2ヶ月 ※短期解約違約金有',
 'フリーレント1ヶ月 ※短期解約違約金有',
 'フリーレント2ヶ月 1年未満解約時違約金あり',
 'フリーレント1ヶ月 詳細はお問合せ下さいませ',
 'フリーレント1ヶ月 1年未満解約時違約金有',
 'フリーレント1ヶ月 １年未満の解約は短期解約違約金あり',
 'フリーレント1ヶ月 フリーレント一か月',
 'フリーレント1ヶ月 １年未満解約違約金有',
 'フリーレント1ヶ月 2022年10月31日までの申込みに適用します。',
 'フリーレント2ヶ月 10月末日まで、1年未満解約時違約金有',
 'フリーレント1ヶ月 1年未満解約時違約金：総賃料の1ヶ月分+フリーレント相当額',
 'フリーレント2ヶ月 2ヶ月',
 'フリーレント1ヶ月 詳細はお問い合わせください。',
 'フリーレント1ヶ月 1ヶ月',
 'フリーレント1ヶ月 1年未満解約時賃料等1ヶ月分の違約金有']

In [37]:
# 条件数変数

condition = suumo_details['条件'] == '-'
suumo_details['条件数'] = 0
suumo_details.loc[condition,'条件数'] = 0
suumo_details.loc[~condition,'条件数'] = suumo_details.loc[~condition,'条件'].map(lambda x: len(x.split('/')))

In [ ]:
# 取り扱い店舗物件コードはよくわからなかった

In [ ]:
# SUUMO物件コードは実質的なユニークキーとして使える？

In [449]:
# 総戸数変数
# 「-」の扱いに困る
condition = suumo_details['総戸数'] == '-'
suumo_details.loc[condition,'url'].tolist()

['https://suumo.jp/chintai/jnc_000046696863/?bc=100294945613',
 'https://suumo.jp/chintai/jnc_000077689180/?bc=100302064363',
 'https://suumo.jp/chintai/jnc_000077692991/?bc=100302075201',
 'https://suumo.jp/chintai/jnc_000077594917/?bc=100301504163',
 'https://suumo.jp/chintai/jnc_000077702981/?bc=100298383398',
 'https://suumo.jp/chintai/jnc_000077560890/?bc=100299735569',
 'https://suumo.jp/chintai/jnc_000077610602/?bc=100300085945',
 'https://suumo.jp/chintai/jnc_000077284341/?bc=100302017244',
 'https://suumo.jp/chintai/jnc_000077313315/?bc=100299794570',
 'https://suumo.jp/chintai/jnc_000077689183/?bc=100302056578',
 'https://suumo.jp/chintai/jnc_000077693008/?bc=100302075083',
 'https://suumo.jp/chintai/jnc_000077686230/?bc=100301998424',
 'https://suumo.jp/chintai/jnc_000077429070/?bc=100300476017',
 'https://suumo.jp/chintai/jnc_000077638638/?bc=100302061950',
 'https://suumo.jp/chintai/jnc_000037143890/?bc=100296426755',
 'https://suumo.jp/chintai/jnc_000077560884/?bc=1003012

In [38]:
# オプション数変数
suumo_details['オプション数'] = suumo_details['オプション'].map(lambda x: len(x.strip().split('、')))

In [85]:
# オプション変数
# 切り分けて配列の状態でしまっておく
# 配列型での持ち運びが出来ないっぽい
# suumo_details['オプション'] = suumo_details['オプション'].map(lambda x: x.strip().split('、'))

In [501]:
# めぼしいオプションは？
# 他の項目見ればよくね？ってのは基本排除した
# オプションの項目にこれだけ押し込むのか…
options = [
    'バストイレ別','室内洗濯置','エアコン','システムキッチン','オートロック',
    '浴室乾燥機','洗面所独立','温水洗浄便座','TVインターホン','バルコニー',
    '宅配ボックス','都市ガス','フローリング','シューズボックス','クロゼット',
    '駐輪場','エレベーター','敷地内ゴミ置き場','防犯カメラ','ガスコンロ対応',
    '保証人不要','追炊機能浴室','洗面化粧台','脱衣所','前居室フローリング',
    '2口コンロ','初期費用カード決済可','光ファイバー','BS・CS','24時間換気システム',
    '角住戸','ネット使用料不要','グリル付','3口以上コンロ','ペット相談','24時間ゴミ出し可',
    '外壁タイル張り','CATV ','CS','ディンプルキー','ダブルロックキー','保証会社利用可',
    'オートバス','高速ネット対応','洗面所にドア','二人入居相談','全居室洋室',
    'バイク置場','2面採光','シャワー付洗面台','陽当り良好','照明付','ネット専用回線',
    '南面バルコニー','ウォークインクロゼット','駅まで平坦','耐震構造','通風良好',
    '対面式キッチン','全居室収納','日勤管理','閑静な住宅地','玄関収納','ガスレンジ付',
    '人感照明センサー','南面リビング','エアコン全室','平坦地','LDK12畳以上','耐火構造',
    'デザイナーズ','クロゼット2ヶ所','眺望良好','独立型キッチン','フリーレント','LAN',
    '分譲賃貸','セキュリティ会社加入済','IHクッキングヒーター','最上階','高層階','未入居',
    '家賃カード決済可','カードキー','室内物干機','楽器相談','タワー型マンション',
    'CATVインターネット','学生相談','24時間緊急通報システム','シューズWIC','浄水器',
    '複層ガラス','床暖房','シャッター','始発駅','フロントサービス','緑豊かな住宅地',
    'オープンキッチン','駅前','ペット専用設備','食器洗乾燥機','ワイドバルコニー',
    '2WAYバルコニー','テラス','2面バルコニー','和室','ディスポーザー','プロパンガス',
    'オール電化','家具付','ルーフバルコニー','ロフト','','',
]
option_list = []
for i in range(suumo_details.shape[0]):
    option_room = suumo_details.loc[i,'オプション'].strip().split('、')
    for option in option_room:
        option_list.append(option)
pd.DataFrame(option_list)[0].value_counts()[300:350]

LDK18畳以上     1
2階洗面台        1
クッションフロア     1
テレビ          1
ハイサッシ        1
バリアフリー       1
AED付         1
路面電車沿線       1
内装リフォーム後渡    1
Name: 0, dtype: int64

In [34]:
# 保存する変数一覧
features = [
    '物件名', '家賃', '管理費・共益費', '敷金', '礼金', '保証金', '敷引・償却', 
    '都道府県', '市区町村', '市区町村以下', '経度', '緯度', '皇居からの距離', 
    '駅徒歩', '路線', '駅', '歩', 'バス', '車', '最寄駅からの距離', 
    '間取り', '部屋数', '間取り_S', '間取り_L', '間取り_D', '間取り_K', 
    '面積', '築年数', '階数', '地下', '地上', '階', '向き', '建物種別', 
    'オプション','オプション数','構造', '駐車場有無', '駐車場距離', '駐車場料金',
    '入居', '取引態様', '条件','条件数','SUUMO物件コード', 'url', 
]

In [113]:
suumo_details[features]

,物件名,家賃,管理費・共益費,敷金,礼金,保証金,敷引・償却,都道府県,市区町村,市区町村以下,...,構造,駐車場有無,駐車場距離,駐車場料金,入居,取引態様,条件,条件数,SUUMO物件コード,url
0,プチ・ボヌール,14.8,5000,14.8,14.8,0.0,0.0,東京都,大田区,南蒲田３,...,木造,0,0,0,即,貸主,-,0,100294945613,https://suumo.jp/chintai/jnc_000046696863/?bc=...
1,ポルトボヌール,11.5,3000,0.0,11.5,0.0,0.0,東京都,荒川区,東尾久６,...,木造,0,0,0,待ち,仲介,二人入居可,1,100302064363,https://suumo.jp/chintai/jnc_000077689180/?bc=...
2,日暮里・舎人ライナー 熊野前駅 3階建 築6年,11.5,3000,0.0,11.5,0.0,0.0,東京都,荒川区,東尾久６,...,木造,0,0,0,待ち,仲介,-,0,100302075201,https://suumo.jp/chintai/jnc_000077692991/?bc=...
3,Ｃｒｅｓｔ Ｌｅ Ｃｅｉｌ,14.4,8000,14.4,14.4,0.0,0.0,東京都,荒川区,西尾久２,...,鉄骨,0,0,0,待ち,仲介,二人入居可/子供可/ルームシェア不可,3,100302058292,https://suumo.jp/chintai/jnc_000077702975/?bc=...
4,Ｃｒｅｓｔ Ｌｅ Ｃｅｉｌ,14.5,8000,14.5,14.5,0.0,0.0,東京都,荒川区,西尾久２,...,鉄骨,0,0,0,待ち,仲介,二人入居可/子供可/ルームシェア不可,3,100302058345,https://suumo.jp/chintai/jnc_000077702976/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196088,東武伊勢崎線 梅島駅 2階建 築35年,4.5,1000,0.0,0.0,0.0,0.0,東京都,足立区,梅田５,...,木造,0,0,0,即,代理,子供不可/事務所利用不可/ルームシェア不可,3,100276133359,https://suumo.jp/chintai/jnc_000077666436/?bc=...
196089,東京メトロ半蔵門線 住吉駅 3階建 築38年,9.0,3000,9.0,0.0,0.0,0.0,東京都,江東区,千田,...,鉄骨,0,0,0,相談,仲介,-,0,100291602676,https://suumo.jp/chintai/jnc_000069250547/?bc=...
196090,ルーブル亀戸天神前,8.0,9000,0.0,8.0,0.0,0.0,東京都,江東区,亀戸３,...,鉄筋コン,0,0,0,待ち,代理,単身者限定/子供不可/事務所利用不可/ルームシェア不可,4,100301012977,https://suumo.jp/chintai/jnc_000077523861/?bc=...
196091,エトワールブランシュ新橋,11.4,6000,0.0,11.4,0.0,0.0,東京都,港区,新橋４,...,鉄筋コン,1,0,38500,待ち,仲介,単身者可,1,100294075873,https://suumo.jp/chintai/jnc_000076124149/?bc=...


In [49]:
# csvに保存
suumo_details[features].to_csv('./data/csv/SuumoDetailsFinal20221108.csv',index=False)

In [50]:
# 呼び出して確認
suumo_details = pd.read_csv('./data/csv/SuumoDetailsFinal20221108.csv')
suumo_details

,物件名,家賃,管理費・共益費,敷金,礼金,保証金,敷引・償却,都道府県,市区町村,市区町村以下,...,構造,駐車場有無,駐車場距離,駐車場料金,入居,取引態様,条件,条件数,SUUMO物件コード,url
0,プチ・ボヌール,14.8,5000,14.8,14.8,0.0,0.0,東京都,大田区,南蒲田３,...,木造,0,0,0,即,貸主,-,0,100294945613,https://suumo.jp/chintai/jnc_000046696863/?bc=...
1,ポルトボヌール,11.5,3000,0.0,11.5,0.0,0.0,東京都,荒川区,東尾久６,...,木造,0,0,0,待ち,仲介,二人入居可,1,100302064363,https://suumo.jp/chintai/jnc_000077689180/?bc=...
2,日暮里・舎人ライナー 熊野前駅 3階建 築6年,11.5,3000,0.0,11.5,0.0,0.0,東京都,荒川区,東尾久６,...,木造,0,0,0,待ち,仲介,-,0,100302075201,https://suumo.jp/chintai/jnc_000077692991/?bc=...
3,Ｃｒｅｓｔ Ｌｅ Ｃｅｉｌ,14.4,8000,14.4,14.4,0.0,0.0,東京都,荒川区,西尾久２,...,鉄骨,0,0,0,待ち,仲介,二人入居可/子供可/ルームシェア不可,3,100302058292,https://suumo.jp/chintai/jnc_000077702975/?bc=...
4,Ｃｒｅｓｔ Ｌｅ Ｃｅｉｌ,14.5,8000,14.5,14.5,0.0,0.0,東京都,荒川区,西尾久２,...,鉄骨,0,0,0,待ち,仲介,二人入居可/子供可/ルームシェア不可,3,100302058345,https://suumo.jp/chintai/jnc_000077702976/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196088,東武伊勢崎線 梅島駅 2階建 築35年,4.5,1000,0.0,0.0,0.0,0.0,東京都,足立区,梅田５,...,木造,0,0,0,即,代理,子供不可/事務所利用不可/ルームシェア不可,3,100276133359,https://suumo.jp/chintai/jnc_000077666436/?bc=...
196089,東京メトロ半蔵門線 住吉駅 3階建 築38年,9.0,3000,9.0,0.0,0.0,0.0,東京都,江東区,千田,...,鉄骨,0,0,0,相談,仲介,-,0,100291602676,https://suumo.jp/chintai/jnc_000069250547/?bc=...
196090,ルーブル亀戸天神前,8.0,9000,0.0,8.0,0.0,0.0,東京都,江東区,亀戸３,...,鉄筋コン,0,0,0,待ち,代理,単身者限定/子供不可/事務所利用不可/ルームシェア不可,4,100301012977,https://suumo.jp/chintai/jnc_000077523861/?bc=...
196091,エトワールブランシュ新橋,11.4,6000,0.0,11.4,0.0,0.0,東京都,港区,新橋４,...,鉄筋コン,1,0,38500,待ち,仲介,単身者可,1,100294075873,https://suumo.jp/chintai/jnc_000076124149/?bc=...


In [51]:
suumo_details.columns

Index(['物件名', '家賃', '管理費・共益費', '敷金', '礼金', '保証金', '敷引・償却', '都道府県', '市区町村',
       '市区町村以下', '経度', '緯度', '皇居からの距離', '駅徒歩', '路線', '駅', '歩', 'バス', '車',
       '最寄駅からの距離', '間取り', '部屋数', '間取り_S', '間取り_L', '間取り_D', '間取り_K', '面積',
       '築年数', '階数', '地下', '地上', '階', '向き', '建物種別', 'オプション', 'オプション数', '構造',
       '駐車場有無', '駐車場距離', '駐車場料金', '入居', '取引態様', '条件', '条件数', 'SUUMO物件コード',
       'url'],
      dtype='object')

In [23]:
suumoA['敷引・償却'] = suumoA['敷引・償却'].replace('実費','0')

In [26]:
suumoA['敷引・償却'] = suumoA['敷引・償却'].astype(float)

In [28]:
suumoA.to_csv('./data/csv/SuumoDetails20221104.csv',index=False)

In [32]:
suumo_details = pd.concat([suumoA,suumoB])